In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

In [ ]:
import random
import json


s = open("/content/gdrive/MyDrive/ColabNotebooks/NCSA/gpt-3_semantic_search/1_top_quality.json")
first = json.load(s)
s = open("/content/gdrive/MyDrive/ColabNotebooks/NCSA/gpt-3_semantic_search/2_decent_enough_to_keep.json")
second = json.load(s)
s = open("/content/gdrive/MyDrive/ColabNotebooks/NCSA/gpt-3_semantic_search/3_to_delete.json")
third = json.load(s)
s = open("/content/gdrive/MyDrive/ColabNotebooks/NCSA/gpt-3_semantic_search/4_invalid_items.json")
fourth = json.load(s)
        
bad_data = []
for dataset in [third, fourth]:
  for row in dataset:
      bad_data.append(row['GPT-3-Semantic-Search-Generations']['answer'])

# create tirplets+ of <question, good answer (pos), and 3 bad answers (neg1, neg2, neg3)>
dev_qa = []
dev_question = []
dev_answer = []
dev_neg = []
for dataset in [second]:
    for row in dataset:
        neg = random.choice(bad_data)
        while any(neg_ex in row['GPT-3-Semantic-Search-Generations']['answer'] for neg_ex in [neg]):
          neg = random.choice(bad_data)
        dev_question.append(row['GPT-3-Semantic-Search-Generations']['question'])
        dev_answer.append(row['GPT-3-Semantic-Search-Generations']['answer'])
        dev_neg.append(neg)
        dev_qa.append([row['GPT-3-Semantic-Search-Generations']['question'], row['GPT-3-Semantic-Search-Generations']['answer'], neg])

train = []
for dataset in [first]:
    for row in dataset:
        neg = random.choice(bad_data)
        while any(neg_ex in row['GPT-3-Semantic-Search-Generations']['answer'] for neg_ex in [neg]):
          neg = random.choice(bad_data)
        train.append([row['GPT-3-Semantic-Search-Generations']['question'], row['GPT-3-Semantic-Search-Generations']['answer'], neg])

In [ ]:
for i in range(len(train)):
  train[i][0] = train[i][0].replace('\\n', '')
  train[i][1] = train[i][1].replace('\n', '')
  train[i][2] = train[i][2].replace('\n', '')
for i in range(len(dev_qa)):
  dev_qa[i][0] = dev_qa[i][0].replace('\\n', '')
  dev_qa[i][1] = dev_qa[i][1].replace('\n', '')
  dev_qa[i][2] = dev_qa[i][2].replace('\n', '')
  dev_question[i] = dev_question[i].replace(']', '')

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from sentence_transformers import evaluation
from torch.utils.data import DataLoader
from sentence_transformers import evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
train_examples = []
for i in range(len(train)):
  train_examples.append(InputExample(texts=[train[i][0], train[i][1], train[i][2]]))

train_dataloader = DataLoader(train_examples, shuffle=False, batch_size=4)
train_loss = losses.TripletLoss(model=model)
evaluator = evaluation.TripletEvaluator(dev_question, dev_answer, dev_neg, 
                                        show_progress_bar = True, name = 'triplet-loss')

model.fit(train_objectives=[(train_dataloader, train_loss)], evaluator = evaluator, epochs=10,
          evaluation_steps=12, 
          output_path = './tripletmodel') 

In [ ]:

from sentence_transformers import SentenceTransformer, util

query = "What is the value of HOLD when the counter is counting?"
docs = ["The value of HOLD when the counter is counting is 0. The value of HOLD when the counter is halted is 1.", "S_2 is 1 when the counter is in the HALT A state and HOLD=1. This is because when S_2=1, the counter is halted, and when HOLD=1, the counter should retain its current value."]

#Load the model
model = SentenceTransformer('/content/gdrive/MyDrive/ColabNotebooks/NCSA/tripletmodel')

evaluator = evaluation.TripletEvaluator(dev_question, dev_answer, dev_neg, 
                                        show_progress_bar = True, name = 'triplet-loss')

evaluator(model)


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

0.967741935483871